In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, precision_score, recall_score

from sys import path

path.append("../analysis/utils/")

from utils import get_datasets
from local_utils import build_clf_beans

In [ ]:
DATASETS = ["webkb"]
CLFS = ["kpr", "ktr", "lpr", "ltr", "sfr", "stmk", "xfr", "xpr", "xtr", "kfr", "ktmk", "lfr", "ltmk", "spr", "str", "xlnet_softmax", "xtmk", "rep_bert"]

In [ ]:
pd_path = "/home/welton/data/pd_datasets/__dset__.csv"
pd_datasets = get_datasets(DATASETS, path=pd_path, sep=';')

In [ ]:
def load_stacking_probs(dataset: str, CLFS: list, train_test: str = "train"):

    probs = {}
    for clf in CLFS:
        probs[clf] = {}
        for fold in np.arange(10):
            probs[clf][fold] = {}
            prob_dir = f"/home/welton/data/clfs_output/split_10/{dataset}/10_folds/{clf}/{fold}/"
            if train_test == "train":
                file_path = f"{prob_dir}/train.npz"
                probs[clf][fold]["train"] = np.load(file_path)[f"X_tain"]
            elif train_test == "test":
                file_path = f"{prob_dir}/test.npz"
                probs[clf][fold]["test"] = np.load(file_path)[f"X_test"]
            else:
                file_path = f"{prob_dir}/train.npz"
                probs[clf][fold]["train"] = np.load(file_path)[f"X_train"]
                file_path = f"{prob_dir}/test.npz"
                probs[clf][fold]["test"] = np.load(file_path)[f"X_test"]
    return probs

In [ ]:
probs = load_stacking_probs("webkb", CLFS, "train_test")

In [49]:
for fold in np.arange(10):
    for clf in ["lfr"]:
        df = pd_datasets["webkb"]
        test = df[df.fold_id == fold]
        train = df[df.fold_id != fold]
        conf_freq, hit_counts = build_clf_beans(probs[clf][fold]["train"], train.label.values)
        hits_by_conf = { key: hit_counts[key] / conf_freq[key] if key in hit_counts else 0 for key in conf_freq }

In [50]:
hits_by_conf

{0.5: 0.2878787878787879,
 0.9: 0.2930417495029821,
 0.4: 0.2254335260115607,
 0.8: 0.28783783783783784,
 0.7: 0.2617702448210923,
 0.6: 0.2808988764044944,
 0.3: 0.22807017543859648,
 0.2: 0.1}

In [32]:
hit_counts

{0.9: 668, 0.8: 17, 0.7: 14, 0.6: 10, 0.5: 5}

In [33]:
conf_freq

{0.9: 737, 0.8: 25, 0.7: 25, 0.6: 17, 0.5: 10, 0.4: 3}